In [1]:
# %%capture
# get_ipython().run_cell_magic('bash', '', 'pip install psycopg2-binary\npip install workalendar\npip install openpyxl\npip install pydot')

In [2]:
import pandas as pd
# import geopandas as gpd
import numpy as np
import os
import glob
#from sqlalchemy import create_engine
#import env

# from pyspark.sql import SparkSession
# from pyspark.sql.functions import substring, length, col, expr
# from pyspark.sql.types import *

import requests

from datetime import datetime, timedelta, date
import time
import pytz
from workalendar.europe import Netherlands

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_error

import helpers_gvb_reworked_v2 as h

import importlib   # to reload helpers without restarting kernel: importlib.reload(h)

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
from time import time

import warnings
import glob

In [3]:
# knmi_df = h.get_knmi_data('knmi/knmi/2021/**/**/*.json.gz')
# knmi_df

#
# import json
# import gzip
#
# json_obj_list = []
# for filepath in glob.glob('knmi/knmi/2021/**/**/*.json.gz'):
#     if not os.path.isfile(filepath) or not os.path.getsize(filepath) > 0:
#         continue
#     # print(filepath)
#     with gzip.open(filepath, 'r') as fin:
#         json_obj_list.extend([json.loads(json_obj_str) for json_obj_str in fin])
#
# knmi_df = pd.DataFrame.from_records(json_obj_list)
#
# knmi_df
# return knmi_df

In [4]:
# stations to create predictions for
stations = ['Centraal Station', 'Station Zuid']

#change every week
week_no = 48

In [5]:
#covid_url = 'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/2020-09-01/2021-07-12' # change 

today = pd.to_datetime("today")
today_str = str(today.year) + "-" + str(today.month) + "-" + str(today.day)
covid_url = 'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/2020-09-01/' + today_str

# Main

## 1. Get data

In [6]:
print('Start loading raw data') 

Start loading raw data


In [7]:
# spark = SparkSession \
#     .builder \
#     .getOrCreate()

In [8]:
t1 = time()

herkomst_2020 = h.get_gvb_data ('Datalab_Reis_Herkomst_Uur_')
bestemming_2020 = h.get_gvb_data ('Datalab_Reis_Bestemming_Uur_')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 125.38961338996887 sec.


In [9]:
t1 = time()

herkomst_2021 = h.get_gvb_data ('Datalab_Reis_Herkomst_Uur_2021')
bestemming_2021 = h.get_gvb_data ('Datalab_Reis_Bestemming_Uur_2021')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 1.519723892211914 sec.


In [10]:
t1 = time()

knmi_obs = h.get_knmi_data('knmi/knmi-observations/2021/**/**/*')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 1.4890871047973633 sec.


In [11]:
t1 = time()

knmi_preds = h.get_knmi_data('knmi/knmi/2021/**/**/*.json.gz')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 45.796528816223145 sec.


In [12]:
# knmi_obs = knmi_obs.toPandas()

In [13]:
# knmi_preds = knmi_preds.toPandas()

In [14]:
covid_df_raw = pd.DataFrame(requests.get(url = covid_url).json()['data'])

In [15]:
holidays_data_raw = Netherlands().holidays(2019) + Netherlands().holidays(2020) + Netherlands().holidays(2021) 

In [16]:
vacations_df = h.get_vacations()

In [17]:
# events = pd.read_excel('events_zuidoost.xlsx', sheet_name='Resultaat', header=1)
#
# # Clean
# events.dropna(how='all', inplace=True)
# events.drop(events.loc[events['Datum']=='Niet bijzonder evenementen zijn hierboven niet meegenomen.'].index, inplace=True)
# events.drop(events.loc[events['Locatie'].isna()].index, inplace=True)
# events.drop(events.loc[events['Locatie']=='Overig'].index, inplace=True)
# events['Datum'] = events['Datum'].astype('datetime64[ns]')
#
# # Fix location names
# events['Locatie'] = events['Locatie'].apply(lambda x: x.strip()) # Remove spaces
# events['Locatie'] = np.where(events['Locatie'] == 'Ziggo dome', 'Ziggo Dome', events['Locatie'])
# events['Locatie'] = np.where(events['Locatie'] == 'Ziggo Dome (2x)', 'Ziggo Dome', events['Locatie'])
#
# # Get events from 2019 from static file
# events = events[events['Datum'].dt.year>=2019].copy()
# events.reset_index(inplace=True)
# events.drop(columns=['index'], inplace=True)
# events
#
# # Add 2020-present events manually
# events = events.append({'Datum':datetime(2020, 1, 19)}, ignore_index=True) # Ajax - Sparta
# events = events.append({'Datum':datetime(2020, 2, 2)}, ignore_index=True) # Ajax - PSV
# events = events.append({'Datum':datetime(2020, 2, 16)}, ignore_index=True) # Ajax - RKC
# events = events.append({'Datum':datetime(2020, 1, 3)}, ignore_index=True) # Ajax - AZ
#
# # Euro 2021
# events = events.append({'Datum':datetime(2021, 6, 13)}, ignore_index=True) # EURO 2020 Nederland- Oekraïne
# events = events.append({'Datum':datetime(2021, 6, 17)}, ignore_index=True) # EURO 2020 Nederland- Oostenrijk
# events = events.append({'Datum':datetime(2021, 6, 21)}, ignore_index=True) # EURO 2020 Noord-Macedonië - Nederland
# events = events.append({'Datum':datetime(2021, 6, 26)}, ignore_index=True) # EURO 2020 Wales - Denemarken
#
# # Add normalized number of visitors
# max_num_visitors_per_day = events.groupby(['Datum'])['Aantal bezoekers'].sum().max()
# events['visitors_normalized'] = events['Aantal bezoekers'] / max_num_visitors_per_day

In [18]:
events = h.get_events()

In [19]:
def read_csv_dir(dir):
    
    read_csv_beta = pd.read_csv(dir,sep=',')
    
    return read_csv_beta

In [20]:
# t1 = time()
#
# bestemming_2021_csv = pd.concat(map(h.read_csv_dir, glob.glob("data_bestemming/*.csv")))
# herkomst_2021_csv = pd.concat(map(h.read_csv_dir, glob.glob("data_herkomst/*.csv")))
#
# t2 = time()
# print('Completed in %s sec.' % (str(t2 - t1)))

### 2. Prepare data

#### Pre-process data sources

In [21]:
print('Start pre-processing data')

Start pre-processing data


In [22]:
herkomst = pd.concat([herkomst_2020, herkomst_2021])
bestemming = pd.concat([bestemming_2020, bestemming_2021])

In [23]:
# Cast 'AantalReizen' to int to sum up
bestemming['AantalReizen'] = bestemming['AantalReizen'].astype(int)
herkomst['AantalReizen'] = herkomst['AantalReizen'].astype(int)

# Remove all duplicates
bestemming.drop_duplicates(inplace=True)
herkomst.drop_duplicates(inplace=True)

# Group by station name because we are analysing per station
bestemming_grouped = bestemming.groupby(['Datum', 'UurgroepOmschrijving (van aankomst)', 'AankomstHalteNaam'], as_index=False)['AantalReizen'].sum()
herkomst_grouped = herkomst.groupby(['Datum', 'UurgroepOmschrijving (van vertrek)', 'VertrekHalteNaam'], as_index=False)['AantalReizen'].sum()

In [24]:
bestemming_herkomst = h.merge_bestemming_herkomst(bestemming_grouped, herkomst_grouped)

In [25]:
gvb_dfs = []

for station in stations:
    
    gvb_dfs.append(h.preprocess_gvb_data_for_modelling(bestemming_herkomst, station))

In [26]:
knmi_historical = h.preprocess_knmi_data_hour(knmi_obs)

In [27]:
knmi_forecast = h.preprocess_metpre_data(knmi_preds)

In [28]:
covid_df = h.preprocess_covid_data(covid_df_raw)

In [29]:
holiday_df = h.preprocess_holiday_data(holidays_data_raw)

#### Merge datasources

In [30]:
# for df in gvb_dfs:
#     print(df['datetime'])

In [31]:
gvb_dfs_merged = []

for df in gvb_dfs:
    gvb_dfs_merged.append(h.merge_gvb_with_datasources(df, knmi_historical, covid_df, holiday_df, vacations_df, events))

In [89]:
# for df in gvb_dfs_merged:
#     print(df['planned_event'].head(100))
#     df['planned_event'] = np.where(df['planned_event'] > 0, 1, 0)

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
        ...   
95    0.113654
96    0.113654
97    0.113654
98    0.113654
99    0.113654
Name: planned_event, Length: 100, dtype: float64
0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
        ...   
95    0.113654
96    0.113654
97    0.113654
98    0.113654
99    0.113654
Name: planned_event, Length: 100, dtype: float64


In [33]:
# gvb_dfs_merged[0][gvb_dfs_merged[0]['planned_event'] != 0]

,check-outs,check-ins,datetime,hour,week,month,year,weekday,hour_0,hour_1,...,ice,temperature_min,radiation_duration,weather_index,dew_point_temperature,precipitation_h,stringency,holiday,vacation,planned_event
47,1937.0,3374.0,2019-01-03,22,1,1,2019,3,0,0,...,0.0,NaN,0.0,5.0,2.1,0.0,0.0,0,1,0.10655
48,441.0,922.0,2019-01-03,0,1,1,2019,3,1,0,...,0.0,NaN,0.0,5.0,0.0,0.0,0.0,0,1,0.10655
49,24.0,159.0,2019-01-03,1,1,1,2019,3,0,1,...,0.0,NaN,0.0,5.0,1.1,0.0,0.0,0,1,0.10655
50,21.0,30.0,2019-01-03,3,1,1,2019,3,0,0,...,0.0,NaN,0.0,5.0,0.8,0.0,0.0,0,1,0.10655
51,12.0,NaN,2019-01-03,4,1,1,2019,3,0,0,...,0.0,NaN,0.0,7.0,0.9,-0.1,0.0,0,1,0.10655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,3778.0,4064.0,2020-02-16,13,7,2,2020,6,0,0,...,0.0,NaN,0.0,7.0,11.1,-0.1,0.0,0,1,0.50000
9664,4157.0,3446.0,2020-02-16,12,7,2,2020,6,0,0,...,0.0,NaN,0.0,7.0,11.6,0.3,0.0,0,1,0.50000
9665,4028.0,2810.0,2020-02-16,11,7,2,2020,6,0,0,...,0.0,11.7,0.0,7.0,11.5,0.6,0.0,0,1,0.50000
9666,2850.0,1945.0,2020-02-16,10,7,2,2020,6,0,0,...,0.0,NaN,0.0,7.0,11.7,0.6,0.0,0,1,0.50000


### 3. Clean data

In [34]:
print('Start cleaning data')

Start cleaning data


#### Interpolate missing data

In [35]:
gvb_dfs_interpolated = []

for df in gvb_dfs_merged:
    gvb_dfs_interpolated.append(h.interpolate_missing_values(df))

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\notebooks_predictions\gvb_week\helpers_gvb_reworked_v2.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  checkins_interpolator.fit(X_train, y_train)
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\notebooks_predictions\gvb_week\helpers_gvb_reworked_v2.py:536: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  checkouts_interpolator.fit(X_train, y_train)
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\notebooks_predictions\gvb_week\helpers_gvb_reworked_v2.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  checkins_interpolator.fit(X_train, y_train)
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\notebooks_predictio

In [36]:
# gvb_dfs_interpolated[0][gvb_dfs_interpolated[0]['planned_event'] != 0]

,check-outs,check-ins,datetime,hour,week,month,year,weekday,hour_0,hour_1,...,ice,temperature_min,radiation_duration,weather_index,dew_point_temperature,precipitation_h,stringency,holiday,vacation,planned_event
47,1937.0,3374.00,2019-01-03,22,1,1,2019,3,0,0,...,0.0,NaN,0.0,5.0,2.1,0.0,0.0,0,1,0.10655
48,441.0,922.00,2019-01-03,0,1,1,2019,3,1,0,...,0.0,NaN,0.0,5.0,0.0,0.0,0.0,0,1,0.10655
49,24.0,159.00,2019-01-03,1,1,1,2019,3,0,1,...,0.0,NaN,0.0,5.0,1.1,0.0,0.0,0,1,0.10655
50,21.0,30.00,2019-01-03,3,1,1,2019,3,0,0,...,0.0,NaN,0.0,5.0,0.8,0.0,0.0,0,1,0.10655
51,12.0,35.34,2019-01-03,4,1,1,2019,3,0,0,...,0.0,NaN,0.0,7.0,0.9,-0.1,0.0,0,1,0.10655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,3778.0,4064.00,2020-02-16,13,7,2,2020,6,0,0,...,0.0,NaN,0.0,7.0,11.1,-0.1,0.0,0,1,0.50000
9664,4157.0,3446.00,2020-02-16,12,7,2,2020,6,0,0,...,0.0,NaN,0.0,7.0,11.6,0.3,0.0,0,1,0.50000
9665,4028.0,2810.00,2020-02-16,11,7,2,2020,6,0,0,...,0.0,11.7,0.0,7.0,11.5,0.6,0.0,0,1,0.50000
9666,2850.0,1945.00,2020-02-16,10,7,2,2020,6,0,0,...,0.0,NaN,0.0,7.0,11.7,0.6,0.0,0,1,0.50000


In [46]:
gvb_dfs_final = []

for df in gvb_dfs_interpolated:
    
    df['check-ins'] = df['check-ins'].astype(int)
    df['check-outs'] = df['check-outs'].astype(int)
    df[['check-ins_week_ago', 'check-outs_week_ago']] = df.apply(lambda x: h.get_crowd_last_week(df, x), axis=1, result_type="expand")
    
    gvb_dfs_final.append(df)

In [47]:
gvb_dfs_final[0][gvb_dfs_final[0]['planned_event'] != 0]

,check-outs,check-ins,datetime,hour,week,month,year,weekday,hour_0,hour_1,...,radiation_duration,weather_index,dew_point_temperature,precipitation_h,stringency,holiday,vacation,planned_event,check-ins_week_ago,check-outs_week_ago
47,1937,3374,2019-01-03,22,1,1,2019,3,0,0,...,0.0,5.0,2.1,0.0,0.0,0,1,0.10655,2862.125000,1492.750000
48,441,922,2019-01-03,0,1,1,2019,3,1,0,...,0.0,5.0,0.0,0.0,0.0,0,1,0.10655,1165.428571,640.428571
49,24,159,2019-01-03,1,1,1,2019,3,0,1,...,0.0,5.0,1.1,0.0,0.0,0,1,0.10655,673.250000,90.000000
50,21,30,2019-01-03,3,1,1,2019,3,0,0,...,0.0,5.0,0.8,0.0,0.0,0,1,0.10655,243.500000,143.625000
51,12,35,2019-01-03,4,1,1,2019,3,0,0,...,0.0,7.0,0.9,-0.1,0.0,0,1,0.10655,172.125000,78.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9663,3778,4064,2020-02-16,13,7,2,2020,6,0,0,...,0.0,7.0,11.1,-0.1,0.0,0,1,0.50000,2807.000000,2994.000000
9664,4157,3446,2020-02-16,12,7,2,2020,6,0,0,...,0.0,7.0,11.6,0.3,0.0,0,1,0.50000,2588.000000,3375.000000
9665,4028,2810,2020-02-16,11,7,2,2020,6,0,0,...,0.0,7.0,11.5,0.6,0.0,0,1,0.50000,2181.000000,3367.000000
9666,2850,1945,2020-02-16,10,7,2,2020,6,0,0,...,0.0,7.0,11.7,0.6,0.0,0,1,0.50000,1506.000000,2525.000000


#### 4. Create model dataframes

In [67]:
# Define features and targets. This is the same for all stations at the moment.

# Define features and targets. This is the same for all stations at the moment.
features = ['year', 'month', 'weekday', 'hour', 'holiday', 'vacation', 'planned_event',  'stringency', 'temperature', 'wind_speed', 'precipitation_h','global_radiation']

targets = ['check-ins', 'check-outs']

In [68]:
data_splits = []

for df in gvb_dfs_final:
    df = df[['datetime'] + features + targets]

    # print(df['datetime'].max())
    # last_week = pd.date_range(df.datetime.max()-pd.Timedelta(7, unit='D')+pd.DateOffset(1), df['datetime'].max())
    # two_weeks_before = pd.date_range(last_week.min()-pd.Timedelta(7, unit='D'), last_week.min()-pd.DateOffset(1))
    #
    # train = df[df['datetime']<two_weeks_before.min()]
    # validation = df[(df['datetime']>=two_weeks_before.min()) & (df['datetime']<=two_weeks_before.max())]
    # test = df[(df['datetime']>=last_week.min()) & (df['datetime']<=last_week.max())]

    train, validation, test = h.get_train_val_test_split(df.dropna())
    data_splits.append([train, validation, test])

In [69]:
X_train_splits = []
y_train_splits = []

X_validation_splits = []
y_validation_splits = []

X_test_splits = []
y_test_splits = []

for split in data_splits:
    
    X_train_splits.append(split[0][features])
    y_train_splits.append(split[0][targets])
    
    X_validation_splits.append(split[1][features])
    y_validation_splits.append(split[1][targets])
    
    X_test_splits.append(split[2][features])
    y_test_splits.append(split[2][targets])

In [70]:
def get_crowd_last_week(df, row):
    week_ago = row['datetime'] - timedelta(weeks=1)
    subset_with_hour = df[(df['datetime']==week_ago) & (df['hour']==row['hour'])]
    
    # If crowd from last week is not available at exact date- and hour combination, then get average crowd of last week.
    subset_week_ago = df[(df['year']==row['year']) & (df['week']==row['week']) & (df['hour']==row['hour'])]
    
    checkins_week_ago = 0
    checkouts_week_ago = 0
    
    if len(subset_with_hour) > 0: # return crowd from week ago at the same day/time (hour)
        checkins_week_ago = subset_with_hour['check-ins'].mean()
        checkouts_week_ago = subset_with_hour['check-outs'].mean() 
    elif len(subset_week_ago) > 0: # return average crowd the hour group a week ago
        checkins_week_ago = subset_week_ago['check-ins'].mean()
        checkouts_week_ago = subset_week_ago['check-outs'].mean()
        
    return [checkins_week_ago, checkouts_week_ago]

In [71]:
# Dataframes to predict cdfheck-ins and check-outs of next week
X_predict_dfs = []

for df in gvb_dfs_final:
    X_predict_dfs.append(h.get_future_df(features, df, covid_df.tail(1)['stringency'][0], holiday_df, vacations_df, knmi_forecast, events))

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\notebooks_predictions\gvb_week\helpers_gvb_reworked_v2.py:646: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['peak_period'][df.hour.isin([7,8,17,18])] = 1
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\notebooks_predictions\gvb_week\helpers_gvb_reworked_v2.py:646: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['peak_period'][df.hour.isin([7,8,17,18])] = 1


In [72]:
X_train_splits[0]['planned_event'][X_train_splits[0]['planned_event'] != 0]

47      0.10655
48      0.10655
49      0.10655
50      0.10655
51      0.10655
         ...   
9663    0.50000
9664    0.50000
9665    0.50000
9666    0.50000
9667    0.50000
Name: planned_event, Length: 2416, dtype: float64

### 5. Create model

In [73]:
print('Start modelling')

Start modelling


In [74]:
len(data_splits)

2

In [75]:
basic_models = []

for x in range(0, len(data_splits)):
    model_basic, r_squared_basic, mae_basic, rmse_basic = h.train_random_forest_regressor(X_train_splits[x], y_train_splits[x], 
                                                                                          X_validation_splits[x], y_validation_splits[x], 
                                                                                          None)
    basic_models.append([model_basic, r_squared_basic, mae_basic, rmse_basic])

In [76]:
#### Tune (hyper-)parameters (not done because models currently do not improve with hyperparameter tuning)

In [77]:
# Specify hyperparameters, these could be station-specific. For now, default hyperparameter settings are being used.
centraal_station_hyperparameters = None
station_zuid_hyperparameters = None
#station_bijlmer_arena_hyperparameters = 2

hyperparameters = [centraal_station_hyperparameters,
                  station_zuid_hyperparameters
#                  ,station_bijlmer_arena_hyperparameters
                  ]

In [78]:
#tuned_models = []

#for x in range(0, len(data_splits)):
#    model_tuned, r_squared_tuned, mae_tuned, rmse_tuned = h.train_random_forest_regressor(X_train_splits[x], y_train_splits[x], 
#                                                                                          X_validation_splits[x], y_validation_splits[x], 
#                                                                                          hyperparameters[x])
#    tuned_models.append([model_tuned, r_squared_tuned, mae_tuned, rmse_tuned])

##### Improvements compared to basic model (negative is worse performance)

In [79]:
#for x in range(0, len(basic_models)):
#    print("R-squared difference", tuned_models[x][1]-basic_models[x][1])
#    print("MAE difference", tuned_models[x][2]-basic_models[x][2])
#    print("RMSE difference", tuned_models[x][3]-basic_models[x][3])

#### Train test model (including validation data)

In [80]:
test_models = []

for x in range(0, len(data_splits)):
    X_train_with_val = pd.concat([X_train_splits[x], X_validation_splits[x]])
    y_train_with_val = pd.concat([y_train_splits[x], y_validation_splits[x]])
    
    model_test, r_squared_test, mae_test, rmse_test = h.train_random_forest_regressor(X_train_with_val, y_train_with_val, 
                                                                                          X_test_splits[x], y_test_splits[x], 
                                                                                          hyperparameters[x])
    test_models.append([model_test, r_squared_test, mae_test, rmse_test])

In [81]:
test_models

[[RandomForestRegressor(random_state=1),
  0.8897040024265483,
  108.05623188405795,
  159.29458900259996],
 [RandomForestRegressor(random_state=1),
  0.8539510043445495,
  75.96007812500002,
  104.34639016201639]]

#### Check models on R-squared score

In [82]:
for x in range(0, len(test_models)):
    station_name = stations[x]
    r_squared = test_models[x][1]
    if r_squared < 0.7:
        warnings.warn("Model for " + station_name + " shows unexpected performance!")

#### Train final models (to make predictions)

In [83]:
final_models = []

for x in range(0, len(data_splits)):
    X_train_with_val = pd.concat([X_train_splits[x], X_validation_splits[x], X_test_splits[x]])
    y_train_with_val = pd.concat([y_train_splits[x], y_validation_splits[x], y_test_splits[x]])
    
    model_final = h.train_random_forest_regressor(X_train_with_val, y_train_with_val, X_test_splits[x], y_test_splits[x], 
                                                  hyperparameters[x])[0]
    final_models.append(model_final)

In [84]:
print('Start preparing data')

Start preparing data


In [85]:
predictions = []

for predict_df in X_predict_dfs:
    for model in final_models:
        prediction = h.predict(model, predict_df.dropna())
        predictions.append(prediction)

### 6. Save

In [ ]:
#today = pd.to_datetime("today")
#today_str = str(today.year) + str(today.month) + str(today.day)

In [ ]:
predictions[0].to_csv(('output/prediction_all_week_' + str(week_no) + '.csv'))

### 7. Make graphs

In [ ]:
df_best_1 = pd.read_csv("data_bestemming/Datalab_Reis_Bestemming_Uur_"+"{}.csv".format((today - timedelta(days=0)).strftime('%Y%m%d')), sep = ";")
df_best_2 = pd.read_csv("data_bestemming/Datalab_Reis_Bestemming_Uur_"+"{}.csv".format((today - timedelta(days=7)).strftime('%Y%m%d')), sep = ";")
df_best_3 = pd.read_csv("data_bestemming/Datalab_Reis_Bestemming_Uur_"+"{}.csv".format((today - timedelta(days=14)).strftime('%Y%m%d')), sep = ";")
df_best_4 = pd.read_csv("data_bestemming/Datalab_Reis_Bestemming_Uur_"+"{}.csv".format((today - timedelta(days=28)).strftime('%Y%m%d')), sep = ";")

In [ ]:
df_best = h.preprocess_gvb_data(pd.concat([df_best_1, df_best_2, df_best_3, df_best_4]))

In [ ]:
df_best_weekday = df_best[df_best['weekday'].isin([0,1,2,3,4])]
df_best_weekend = df_best[df_best['weekday'].isin([5,6])]

In [ ]:
df_best_weekday = df_best_weekday[df_best_weekday['arrival_stop_name'] == 'Centraal Station']
df_best_weekend = df_best_weekend[df_best_weekend['arrival_stop_name'] == 'Centraal Station']

In [ ]:
df_best_weekday_grouped = df_best_weekday.groupby('week').sum().reset_index()
df_best_weekend_grouped = df_best_weekend.groupby('week').sum().reset_index()

In [ ]:
df_best_weekday_grouped = df_best_weekday_grouped[df_best_weekday_grouped.week.isin([week_no-1, week_no-2, week_no-3, week_no-4])]
df_best_weekend_grouped = df_best_weekend_grouped[df_best_weekend_grouped.week.isin([week_no-1, week_no-2, week_no-3, week_no-4])]

In [ ]:
df_best_weekday_grouped = df_best_weekday_grouped[['week','count']]
df_best_weekend_grouped = df_best_weekend_grouped[['week','count']]

In [ ]:
predictions[0]['week'] = predictions[0]['datetime'].dt.isocalendar().week  

In [ ]:
pred_weekdays = predictions[0][predictions[0].weekday.isin([0,1,2,3,4])]
pred_weekends = predictions[0][predictions[0].weekday.isin([5,6])]

In [ ]:
pred_weekdays = pred_weekdays[['week','check-outs_predicted']].groupby('week').sum().rename(columns={'check-outs_predicted':'count'}).reset_index()
pred_weekends = pred_weekends[['week','check-outs_predicted']].groupby('week').sum().rename(columns={'check-outs_predicted':'count'}).reset_index()

In [ ]:
df_plot_weekday = pd.concat([df_best_weekday_grouped, pred_weekdays])
df_plot_weekend = pd.concat([df_best_weekend_grouped, pred_weekends])

In [ ]:
df_plot_weekday['count'] = df_plot_weekday['count'] / 5
df_plot_weekend['count'] = df_plot_weekend['count'] / 2

In [ ]:
df_plot_weekday["color"] = "blue" 
df_plot_weekday["color"][df_plot_weekday["week"] == week_no] = "lightskyblue"   # Change each week
#df_plot["color"][df_plot["week_year"].isin(["9 ('20)", "13 ('20)", "39 ('20)"])] = "lightblue"  # Change each week

df_plot_weekend["color"] = "blue" 
df_plot_weekend["color"][df_plot_weekend["week"] == week_no] = "lightskyblue"   # Change each week
#df_plot["color"][df_plot["week_year"].isin(["9 ('20)", "13 ('20)", "39 ('20)"])] = "lightblue"  # Change each week


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (11,4), dpi = 100, frameon = False, sharey = True, constrained_layout = True)

ax1.bar(x = df_plot_weekday['week'], height = df_plot_weekday['count'], color = df_plot_weekday['color'])
ax1.set_title('Doordeweeks (ma-vr)')
ax1.set_xlabel("Week")
ax1.set_ylabel("Aantal Reis-uitchecks per dag \n")
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.grid(axis = 'y', color = 'lightgrey')

ax2.bar(x = df_plot_weekend['week'], height = df_plot_weekend['count'], color = df_plot_weekend['color'])
ax2.set_title('Weekend (za-zo)')
ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.grid(axis = 'y', color = 'lightgrey')
ax2.set_xlabel("Week")

gemeten = mpatches.Patch(color='blue', label='Gemeten')
voorspelling = mpatches.Patch(color='lightskyblue', label='Voorspelling')
plt.legend(handles=[gemeten, voorspelling], bbox_to_anchor=(1.2, 0.5, 0.5, 0.5))

ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax2.grid(axis = 'y', color = 'lightgrey')
plt.rcParams["axes.axisbelow"] = True

fig.suptitle('Centraal Station (Metro)\n', fontsize = 18)

my_filename = "output/weekly_report_gvb_prediction_" + today_str + ".png" 
        
plt.savefig(my_filename, bbox_inches='tight')
plt.show()

### 8. Evaluations

In [ ]:
#def read_csv_dir(dir):
    
    
#     fields = ['datetime', 'predict_xg_CMSA-GAKH-01', 'predict_xg_GACM-02', 'predict_xg_CMSA-GAWW-15', 'predict_xg_CMSA-GAWW-14']
#    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z')
#    read_csv_beta = pd.read_csv(dir,sep=','
                                #usecols=fields
#                               )
    
#    return read_csv_beta

In [ ]:
#read csv files from the previous week
#df_pred = pd.concat(map(read_csv_dir, glob.glob("output/prediction_all_week_*.csv")))

In [ ]:
#predicted counts
#df_pred['datetime'] = df_pred['hour'] + + df['datetime']
#df_pred['datetime'] = pd.to_datetime(df_pred['datetime'])
#df_pred = df_pred.set_index('datetime')
#df_pred['week'] = df_pred.index.isocalendar().week
#slices data from only the last 4 weeks
#df_pred = df_pred[df_pred['week'].isin([my_week-1, my_week-2, my_week-3])]
#df_pred = df_pred.groupby('week').sum()